In [371]:
import pandas as pd
import numpy as np
import os

#load odds and results
data = pd.read_csv('odds&results2024.csv')
abvr_match = pd.read_csv('team_abbrv_match.csv')
data = data.merge(abvr_match, left_on='home_team', right_on='team_name')
data = data.merge(abvr_match, left_on='away_team', right_on='team_name')
odds_and_results = data.rename(columns={"team_abbrv_x": "home_team_abbrv", "team_abbrv_y": "away_team_abbrv"})


In [372]:
#parse through raw output and add to full_list dataframe
players = [37,'Billiard91','Bob Z','EllieisAwesome1','JJB','JJapcon','Jack Ballard','K-Dog','Kathy Zwartz','Kkrugz','Linds Domino','Lizzy_Z','Matt McKenna','MattWojtanek','Michael Zwartz','Midway Monsters','Murr-auders','PBW','Shan-diddy','Shaner','Shumarie','TAZ','Team Boylan','TeamGalarza1020','UrBoiJZ','Zapp','abfarias','taz7611']
full_picks = pd.DataFrame() 

#parse all results for clean data file
directory = os.fsencode('weekly_data\picks2024')
    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    nfl_week = (filename.split(".")[0]).split("week")[1]
    
    #df = pd.read_excel(f"weekly_data\picks2024\{filename}")
    df = pd.read_excel(f"weekly_data\picks2024\picks_export_week6.xls")
    df.columns.values[0] = "player" #rename column for merge

    #fill in black player names in points rows
    df.replace(" ", np.nan, inplace=True)
    df['player'] = df['player'].ffill()

    #add 'player' for home team and drop other game rows
    df['player'].fillna(value='home_team', inplace=True)
    df = df.drop(index=[0,1], columns=df.columns[-1])

    #reverse rows and columns
    df1 = df.transpose()

    # Make the first row the new column names
    df1.columns = df1.iloc[0]  # Set the new column names
    df1 = df1[1:]  # Drop the first row
    df1 = df1.reset_index(drop=True)
    df2 = df1.copy()
    df2['nfl_week'] = int(nfl_week)

    for player in players:
        if player in df2.columns:
            player_df = df2[['home_team', 'nfl_week', player]].copy()
            player_df['player'] = player
            player_df.columns.values[2] = "pick"
            player_df.columns.values[3] = "points"
            
            #append each player to the full list        
            full_picks = pd.concat([full_picks, player_df])